# GRAND Data reading -> reconstructrion -> writing with AOI

First, imports. The first is necessary, the rest is just for plotting.

In [ ]:
from grand.aoi import *
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

## Open the list of events

1. The first parameter is GRAND directory or GRAND ROOT file
2. The `use_trawvoltage=True` is necessary for experimental data (where we have trawvoltage instead of voltage)

In [ ]:
el = EventList("confirmed_events", use_trawvoltage=True)

## Loop through the list of events

1. The main information from the ROOT data files is read into the Event class instance
2. Each Event data is stored in subclasses such as antenna, voltages, efields, shower
3. Each subclass has its own members, such as traces
4. Some subclasses are lists, such as voltages or efields, where each index is the number of DU in the event
5. Some subclasses have subclasses, like Voltage has TimeTrace3D, which in turn has x,y,z - this is what we access below

In [ ]:
for i, e in enumerate(el):
    print(f"Run #{e.run_number}, Event #{e.event_number}")
    print("Voltage trace [0], x:", e.voltages[0].trace.x)

## Plotting traces

1. Just plot `e.voltages[i].t_vector, e.voltages[i].trace.x`
2. You can loop through e.voltages to draw different DUs
3. DU ID is stored in `e.voltages[i].du_id`

You need to manually go to the next cell after the plotting loop finishes (quirks of waiting for key press)

In [ ]:
for i, e in enumerate(el):
    print(f"Run #{e.run_number}, Event #{e.event_number}, DU_ID {e.voltages[0].du_id}")
    plt.figure()
    plt.plot(e.voltages[0].t_vector, e.voltages[0].trace.x)
    display(plt.gcf())
    clear_output(wait=True)

    # Wait for key press
    rep = ""
    while not rep in [ 'q', 'Q', "x", "X", "c", "C" ]:
        rep = input('enter "c" or "q" to continue, "x" to quit, "b" to break the loop: ')
        if len(rep)<1: continue
            
    plt.close()

    if i==3: break

## Analytical plane wave reconstruction function

Just for ilustrational purposes. Doesn't always give a proper result.

In [ ]:
def plane_wave_direction_with_errors(x, y, t, sigma=None):
    """
    Analytic plane-wave fit for EAS arrival direction, with 1σ uncertainties.

    Inputs:
      x, y : arrays of detector coords [m]
      t    : array of trigger times [ns]
      sigma: optional array of timing uncertainties [ns];
             if None, all weights = 1.

    Returns:
      theta, phi, theta_err, phi_err, t0, a, b
    """
    # Speed of light [m/ns]
    c = 0.299792458  # m/ns :contentReference[oaicite:0]{index=0}

    # Convert to numpy arrays
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    t = np.asarray(t, float)
    N = len(x)

    t -= np.min(t)

    # Weights: w_i = 1/σ_i^2 or 1
    if sigma is None:
        sigma = np.ones(N)*15

    w = 1.0 / np.square(sigma)  # :contentReference[oaicite:1]{index=1}

    # Centre & scale for stability
    x0 = np.average(x, weights=w)
    y0 = np.average(y, weights=w)
    t0m = np.average(t, weights=w)
    x_c = (x - x0) / c           # include 1/c factor here :contentReference[oaicite:2]{index=2}
    y_c = (y - y0) / c
    t_c = t - t0m                # centred times :contentReference[oaicite:3]{index=3}

    # Build normal-equation sums
    S0  = np.sum(w)
    Sx  = np.sum(w * x_c)
    Sy  = np.sum(w * y_c)
    Sxx = np.sum(w * x_c * x_c)
    Syy = np.sum(w * y_c * y_c)
    Sxy = np.sum(w * x_c * y_c)

    St  = np.sum(w * t_c)
    Sxt = np.sum(w * x_c * t_c)
    Syt = np.sum(w * y_c * t_c)  # :contentReference[oaicite:4]{index=4}

    # Assemble and solve for β = [t0', a, b]
    M = np.array([[S0,  Sx,  Sy],
                  [Sx, Sxx, Sxy],
                  [Sy, Sxy, Syy]])
    v = np.array([St, Sxt, Syt])
    t0p, a, b = np.linalg.solve(M, v)    # :contentReference[oaicite:5]{index=5}

    # Recover the true t0
    t0 = t0p + t0m - (a*x0 + b*y0)/c      # :contentReference[oaicite:6]{index=6}

    # Compute residuals and χ²/ndf for error scaling
    t_pred = t0 + (a*(x - x0) + b*(y - y0)) / c
    resid = (t - t_pred)/sigma
    chi2 = np.sum(w * resid**2)          # :contentReference[oaicite:7]{index=7}
    ndf = N - 3
    if ndf==0: sigma2_res = chi2
    else:
      sigma2_res = chi2 / ndf              # reduced χ² :contentReference[oaicite:8]{index=8}

    # Covariance matrix of [t0', a, b] = M⁻¹ · σ₂_res
    # Cov = np.linalg.inv(M) * sigma2_res  # :contentReference[oaicite:9]{index=9}
    Cov = np.linalg.inv(M)
    var_a = Cov[1,1]
    var_b = Cov[2,2]
    cov_ab = Cov[1,2]                    # :contentReference[oaicite:10]{index=10}

    # Build unit‐normal: n ∝ (-a, -b,  c)
    nx, ny, nz = -a/c, -b/c, 1
    norm = np.sqrt(nx*nx + ny*ny + nz*nz)
    nx, ny, nz = nx/norm, ny/norm, nz/norm

    # Angles
    theta = np.degrees(np.arccos(nz))    # θ ∈ [0°,180°] :contentReference[oaicite:11]{index=11}
    phi   = np.degrees(np.arctan2(ny, nx))  # φ ∈ [-180°,180°] :contentReference[oaicite:12]{index=12}

    # ---------------------------------------------------------------------
    # Propagate uncertainties via ∂θ/∂a, ∂θ/∂b, ∂φ/∂a, ∂φ/∂b
    # ---------------------------------------------------------------------
    R2 = a*a + b*b + 1.0
    # derivatives in radians
    dth_da = -a / (R2 * np.sqrt(R2))
    dth_db = -b / (R2 * np.sqrt(R2))
    dph_da =  b / (a*a + b*b)
    dph_db = -a / (a*a + b*b)

    # variances in degrees^2
    var_theta = (dth_da*180/np.pi)**2 * var_a \
              + 2*(dth_da*180/np.pi)*(dth_db*180/np.pi)*cov_ab \
              + (dth_db*180/np.pi)**2 * var_b
    var_phi   = (dph_da*180/np.pi)**2 * var_a \
              + 2*(dph_da*180/np.pi)*(dph_db*180/np.pi)*cov_ab \
              + (dph_db*180/np.pi)**2 * var_b

    theta_err = np.sqrt(var_theta)
    phi_err   = np.sqrt(var_phi)

    return theta, phi, theta_err, phi_err#, chi2#, sigma2_res#, t0, a, b


## Delete the results directory if already exists

Otherwise the Event will complain and not write out

In [ ]:
import shutil
if Path("reconstructed_events").is_dir():
    shutil.rmtree("reconstructed_events")

## The reconstruction loop

1. Loop through `EventList` like before
2. Insert and execute your reconstruction code for every Event
3. Create the `Shower()` and attach to `e.shower`
4. Fill the `e.shower` with the results of reconstruction
5. Write the `Event` to the directory of your choice

In [ ]:
for i, e in enumerate(el):
    print(f"Run #{e.run_number}, Event #{e.event_number}")

    # *** DIRECTION RECONSTRUCTION ***
    
    # Get the positions of antennas in the current event
    positions = np.array([[ant.position.x[0], ant.position.y[0]] for ant in e.antennas])

    # Get the trigger times (if these are really trigger times is still questionable)
    trigger_times = np.array([v.trigger_time for v in e.voltages])

    # Reduce the number of digits in times - there are Unix times with nanoseconds!
    trigger_times = np.array(trigger_times - np.min(trigger_times)).astype(np.float64)
    
    # Reconstruct the direction with plane wave analytical fit - may fail
    fit_failed = False
    try:
        zenith, azimuth, dzenith, dazimuth = plane_wave_direction_with_errors(positions[:, 0], positions[:, 1], trigger_times)
    except:
        fit_failed = True

    # *** X_MAX "RECONSTRUCTION"
    xmax = np.random.rand()*400+700

    print(f"Reconstruction results: zenith {zenith}, azimuth {azimuth}, Xmax {xmax}")

    # *** SAVING THE RESULTS - only for successful results***

    if fit_failed: continue
    
    # Create the Shower instance
    e.shower = Shower()

    # Fill with reconstruction information
    # We are not storing uncertainties anywhere - we should probably add fields in the trees
    e.shower.zenith = zenith
    e.shower.azimuth = azimuth
    e.shower.Xmax = xmax

    # Write the event in the target directory
    e.write(out_dir="reconstructed_events")
    
    plt.close()

## Source and reconstruction directories contents comparison

In the source directory we have just run information and files with traces

In [ ]:
import os

sorted(os.listdir("confirmed_events"))

1. In the output directory we have the file with reconstructed showers parameters
2. The adc traces file is gone, because (for now) we decided it is not interested for the Event reconstruction purposes (and thus ignored by AOI)

In [ ]:
sorted(os.listdir("reconstructed_events"))

## Reading saved reconstruction results

Same as before, but let's now read "reconstructed_events" directory that we've created above with `EventList`

In [ ]:
el_reco = EventList("reconstructed_events", use_trawvoltage=True)

### Getting all angles and Xmaxes

Loop through the `EventList` as before, and store the parameters in lists

In [ ]:
zeniths, azimuths, xmaxs = [], [], []
for i, e in enumerate(el_reco):
    print(e.run_number, e.event_number, e.voltages[0].trace.x)
    zeniths.append(e.shower.zenith)
    azimuths.append(e.shower.azimuth)
    xmaxs.append(e.shower.Xmax)

### Ploting the read-out reconstructed angles

Simple plotting. You have to look at ROOT output, because I despise matplotlib

In [ ]:
g_pw = ROOT.TGraphPolar(len(zeniths), np.radians(np.array(zeniths)).astype(np.float64), np.array(azimuths).astype(np.float64))
c = ROOT.TCanvas()
g_pw.Draw("A*")
c.Draw()

## Documentation

The **very** outdated documentation is available at http://grand.fuw.edu.pl/analysis_oriented_api/ (l: grand, p: uhenuuhecr), but the main fields of the Event class are there.

**Please test and suggest improvements!**